<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_multiverse_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_multiverse_ai.py

Physics-informed AI pipeline for MultiverseAI:

1. Synthetic dataset of 6 “multiverse” parameters → 3 target metrics
2. 1D normalization (float32)
3. Physics-informed residual enforcing toy domain laws in normalized space
4. MLP with LayerNorm & Dropout for robustness
5. MC-Dropout inference for uncertainty quantification
6. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, early stopping
7. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. Synthetic Multiverse Dataset
# ------------------------------------------------------------------------------
class MultiverseDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        # Features: quantum vacuum fluctuation (QVF), entropy variation (EV),
        # dark energy level (DE), coupling constant (CC),
        # cosmic string density (CSD), inflation rate (IR)
        QVF = np.random.uniform(0.1, 1.0, (n_samples,1))
        EV  = np.random.uniform(0.0, 0.5, (n_samples,1))
        DE  = np.random.uniform(0.1, 2.0, (n_samples,1))
        CC  = np.random.uniform(0.01, 1.0, (n_samples,1))
        CSD = np.random.uniform(0.0, 0.2, (n_samples,1))
        IR  = np.random.uniform(1.0, 10.0, (n_samples,1))

        X_raw = np.hstack([QVF, EV, DE, CC, CSD, IR]).astype(np.float32)

        # Domain formulas for targets:
        # 1. stability = QVF * CC / (DE + ε)
        eps = 1e-6
        stability = QVF * CC / (DE + eps)

        # 2. emergence_prob = exp(-EV * IR)
        emergence_prob = np.exp(-EV * IR)

        # 3. interaction_effect = CSD * CC * DE
        interaction_effect = CSD * CC * DE

        Y_raw = np.hstack([
            stability, emergence_prob, interaction_effect
        ]).astype(np.float32)

        # Add small relative noise
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape).astype(np.float32)

        # Compute 1D normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-6
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-6

        # Normalize
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),
            torch.from_numpy(self.Y[idx])
        )

# ------------------------------------------------------------------------------
# 2. Model Definition
# ------------------------------------------------------------------------------
class MultiverseAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, output_dim=3, p_drop=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p_drop),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(p_drop),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # Denormalize inputs
    X_den = X * stats['X_std'] + stats['X_mean']
    QVF, EV, DE, CC, CSD, IR = X_den.t()

    # Compute true targets
    eps = 1e-6
    stability_t = QVF * CC / (DE + eps)
    emergence_t = torch.exp(-EV * IR)
    interaction_t = CSD * CC * DE

    Y_t = torch.stack([stability_t, emergence_t, interaction_t], dim=1)

    # Normalize true targets
    Y_t_norm = (Y_t - stats['Y_mean']) / stats['Y_std']

    # Compute MSE residual in normalized space
    return nn.MSELoss()(pred, Y_t_norm)

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout for Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    stacked = torch.stack(preds, dim=0)
    return stacked.mean(0), stacked.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop
# ------------------------------------------------------------------------------
def train(model, train_loader, val_loader, stats, device,
          lr=1e-3, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train_loss': [], 'val_loss': []}

    for ep in range(1, epochs+1):
        # Training
        model.train()
        total_train = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred    = model(xb)
            loss, mse, phys = total_loss(pred, yb, xb, stats, lam)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_train += loss.item() * xb.size(0)
        train_loss = total_train / len(train_loader.dataset)

        # Validation
        model.eval()
        total_val = 0.0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred, = (model(xb),)  # just forward
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                total_val += loss.item() * xb.size(0)
        val_loss = total_val / len(val_loader.dataset)

        # Scheduler & logging
        scheduler.step(val_loss)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        print(f"Epoch {ep:03d} | Train {train_loss:.4e} | Val {val_loss:.4e}")

        # Early stopping
        if val_loss < best_val - 1e-6:
            best_val, wait = val_loss, 0
            torch.save(model.state_dict(), "best_multiverse_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {ep}")
                break

    # Load best if available
    if os.path.exists("best_multiverse_ai.pth"):
        model.load_state_dict(torch.load("best_multiverse_ai.pth"))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(hist):
    plt.figure()
    plt.plot(hist['train_loss'], label='Train')
    plt.plot(hist['val_loss'],   label='Val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(title)
    plt.xlabel("True")
    plt.ylabel("Pred")
    plt.tight_layout()
    plt.show()

def plot_uncertainty(model, stats, device):
    # Vary QVF vs DE for stability uncertainty
    QVF_vals = np.linspace(0.1, 1.0, 100, dtype=np.float32)
    DE_vals  = np.linspace(0.1, 2.0, 100, dtype=np.float32)
    Qg, Dg   = np.meshgrid(QVF_vals, DE_vals)
    pts = Qg.size

    # Build tensor grid on device
    grid = torch.zeros((pts, 6), device=device, dtype=torch.float32)
    grid[:, 0] = torch.from_numpy(Qg.ravel()).to(device)
    grid[:, 2] = torch.from_numpy(Dg.ravel()).to(device)
    # fix other features at mean
    for i in (1, 3, 4, 5):
        grid[:, i] = stats['X_mean'][i]

    # normalize
    Xn = (grid - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:, 0].cpu().numpy().reshape(Qg.shape)

    plt.figure(figsize=(6,5))
    plt.pcolormesh(Qg, Dg, U, cmap='viridis', shading='auto')
    plt.colorbar(label="Std Stability")
    plt.xlabel("QVF")
    plt.ylabel("DE")
    plt.title("Uncertainty Heatmap: Stability")
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    import os

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare dataset and stats
    ds = MultiverseDataset(n_samples=6000)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, device=device, dtype=torch.float32),
        'X_std':  torch.tensor(ds.X_std,  device=device, dtype=torch.float32),
        'Y_mean': torch.tensor(ds.Y_mean, device=device, dtype=torch.float32),
        'Y_std':  torch.tensor(ds.Y_std,  device=device, dtype=torch.float32),
    }

    # Split and DataLoaders
    n_val = int(0.2 * len(ds))
    train_ds, val_ds = random_split(ds, [len(ds)-n_val, n_val])
    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=256)

    # Initialize, train, and visualize
    model = MultiverseAI().to(device)
    history = train(
        model, train_loader, val_loader,
        stats, device,
        lr=1e-3, wd=1e-5, lam=1.0,
        epochs=100, patience=10
    )

    plot_history(history)

    # Scatter true vs predicted (denormalized)
    with torch.no_grad():
        X_all = torch.from_numpy(ds.X).to(device)
        Yp_n  = model(X_all).cpu().numpy()
    Yp = Yp_n * ds.Y_std + ds.Y_mean
    Yt = ds.Y * ds.Y_std + ds.Y_mean
    names = ["Stability","Emergence Prob","Interaction Eff"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    # Uncertainty heatmap for stability
    plot_uncertainty(model, stats, device)